In [ ]:
import holoviews as hv
import numpy as np
import pandas as pd
from holoviews import opts
from sklearn.metrics import precision_score, recall_score

from cross import cross_energy_matrix
from curvature import curvature_energy_matrix
from generator import SimpleEventGenerator
from plot import make_tracks_3d
from reconstruct import annealing_curve, update_layer_grad, energy, energy_gradient, should_stop
from segment import gen_segments_all
from total import total_activation_matrix

In [ ]:
N_TRACKS = 10
N_EVENTS = 1

eventgen = SimpleEventGenerator(
    seed=2, field_strength=0.8, noisiness=10, box_size=.5
).gen_many_events(N_EVENTS, N_TRACKS)

hits, track_segments = next(eventgen)
hits

In [ ]:
pos = hits[['x', 'y', 'z']].values

In [ ]:
seg = gen_segments_all(hits)
len(track_segments), len(seg)

In [ ]:
perfect_act = np.zeros(len(seg))
track_segment_set = set(tuple(s) for s in track_segments)
is_in_track = np.array([tuple(s) in track_segment_set for s in seg])
perfect_act[is_in_track] = 1

In [ ]:
ALPHA = 2.2  # forks and joins

BETA = 0  # total activation
DROP_SELF_ACTIVATION_WEIGHTS = True  # in total activation matrix
BIAS = 0.2  # activation bias, instead of total activation matrix

COSINE_POWER = 5
COSINE_MIN = 0.8
DISTANCE_POWER = 0
LZ = hits.groupby('layer').z.mean()
l_dist = (LZ.values[1:] - LZ.values[:-1]).mean()
GAMMA = l_dist ** DISTANCE_POWER
print(GAMMA)

THRESHOLD = 0.5  # activation threshold for segment classification

TMAX = 10
TMIN = 1
ANNEAL_ITERATIONS = 200
STABLE_ITERATIONS = 200

DROPOUT = 0
LEARNING_RATE = 0.1
MIN_ACTIVATION_CHANGE_TO_CONTINUE = 1e-3
SHOULD_STOP_LOOKBACK = 7

In [ ]:
crossing_matrix = cross_energy_matrix(seg) if ALPHA else 0
a, b, c = total_activation_matrix(pos, seg, DROP_SELF_ACTIVATION_WEIGHTS) if BETA else (0, 0, 0)
curvature_matrix = curvature_energy_matrix(pos, seg, COSINE_POWER, COSINE_MIN, DISTANCE_POWER) if GAMMA else 0
e_matrix = ALPHA / 2 * crossing_matrix + BETA / 2 * a - GAMMA / 2 * curvature_matrix
if BETA:
    e_matrix = e_matrix.A  # dense matrix to ndarray for correct .dot behaviour (same as sparse matrix)

In [ ]:
temp_curve = annealing_curve(TMIN, TMAX, ANNEAL_ITERATIONS, STABLE_ITERATIONS)
tcdf = pd.DataFrame({'temp': temp_curve})
tcdf.index.name = 'step'
tcdf.plot()

In [ ]:
act = np.full(len(seg), THRESHOLD * 0.999)
acts = []
for i, t in enumerate(temp_curve):
    acts.append(act.copy())
    grad = energy_gradient(e_matrix, act) + (b if BETA else 0)
    update_layer_grad(act, grad, t, DROPOUT, LEARNING_RATE, BIAS)
    if i > ANNEAL_ITERATIONS and should_stop(act, acts, MIN_ACTIVATION_CHANGE_TO_CONTINUE, SHOULD_STOP_LOOKBACK):
        break

In [ ]:
energy_history = []
for act in acts:
    ef = ALPHA / 2 * energy(crossing_matrix, act) if ALPHA else 0
    en = BETA / 2 * (energy(a, act) + b.dot(act) + c) if BETA else 0
    ec = GAMMA / 2 * energy(curvature_matrix, act) if GAMMA else 0  # inverted to positive
    energy_history.append([ec, en, ef])
energy_history = pd.DataFrame(energy_history, columns=['E_curve', 'E_number', 'E_fork'])
energy_history.index.name = 'step'
energy_history.plot(logy=True)
pass

In [ ]:
small_history = pd.DataFrame([
    (
        precision_score(perfect_act, act>THRESHOLD, zero_division=0),
        recall_score(perfect_act, act>THRESHOLD, zero_division=0),
        act.mean(),
        ((act - perfect_act) ** 2).mean()
    ) for act in acts],
    columns=['precision', 'recall', 'mean_act', 'dist_perfect'])
small_history.index.name = 'step'
small_history.plot()

In [ ]:
n_steps = min(16, len(acts))
steps = np.linspace(0, len(acts) - 1, n_steps, dtype=int)

tracks_3d = []
tracks_projection = []
tracks_by_track = []
for i in steps:
    tp, fp, tn, fn = make_tracks_3d(pos, seg, acts[i], perfect_act, THRESHOLD)
    vdims = ['act', 'perfect_act', 'positive', 'true']
    xyz = hv.Overlay([
        hv.Scatter3D(hits[hits.track == -1], kdims=['x', 'y', 'z'], label='noise', group='hits'),
        hv.Scatter3D(hits[hits.track != -1], kdims=['x', 'y', 'z'], label='hits', group='hits'),
        hv.Path3D(tp, vdims=vdims, label='tp', group='tracks'),
        hv.Path3D(fp, vdims=vdims, label='fp', group='tracks'),
        hv.Path3D(fn, vdims=vdims, label='fn', group='tracks')
    ])
    tracks_3d.append(xyz)

    projection = hv.Points(hits[hits.track != -1], kdims=['x', 'y'], label='hits', group='hits') * \
                 hv.Points(hits[hits.track == -1], kdims=['x', 'y'], label='noise', group='hits') * \
                 hv.Path(tp, kdims=['x', 'y'], vdims=vdims, label='tp', group='tracks') * \
                 hv.Path(fp, kdims=['x', 'y'], vdims=vdims, label='fp', group='tracks') * \
                 hv.Path(fn, kdims=['x', 'y'], vdims=vdims, label='fn', group='tracks')
    tracks_projection.append(projection)

    nodes = hv.Nodes(hits, kdims=['track', 'layer', 'index'])
    no_tn = np.logical_or(acts[i] > THRESHOLD, perfect_act > THRESHOLD)
    graph = hv.Graph(((*seg[no_tn].transpose(), acts[i][no_tn], perfect_act[no_tn]), nodes),
                     vdims=['act', 'perfect_act'])
    tracks_by_track.append(hv.Overlay([graph]))


In [ ]:
hv.extension('matplotlib')
tracks_3d[-1].opts(
    opts.Scatter3D(c='black'),
    opts.Scatter3D('Hits.Noise', c='red'),
    opts.Path3D(color='black', show_legend=True),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(legend_position='right', fig_size=400),
)

In [ ]:
hv.extension('bokeh')
# hv.HoloMap(
#     {s: hv.Histogram(np.histogram(acts[s], bins=32, range=(0,1))) for s in np.linspace(0, len(acts)-1, min(64, len(acts)), dtype=int)},
#     kdims='step', group="Activation", label="All") * \
track_history = \
    hv.HoloMap(
        {s: hv.Histogram(np.histogram(acts[s][perfect_act > THRESHOLD], bins=32, range=(0, 1))) for s in
         np.linspace(0, len(acts) - 1, min(64, len(acts)), dtype=int)},
        kdims='step', group="Activation", label="On track") * \
    hv.HoloMap(
        {s: hv.Histogram(np.histogram(acts[s][perfect_act < THRESHOLD], bins=32, range=(0, 1))) for s in
         np.linspace(0, len(acts) - 1, min(64, len(acts)), dtype=int)},
        kdims='step', group="Activation", label="Off track")

track_history.opts(
    opts.Histogram(width=400, height=400)
)

In [ ]:
hv.extension('plotly')
tracks_3d[-1].opts(
    opts.Scatter3D(size=2, color='black'),
    opts.Scatter3D('Hits.Noise', color='red'),
    opts.Path3D(color='black'),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(width=600, height=600)
)

In [ ]:
hv.extension('matplotlib')
track_history = hv.NdLayout(
    {s: tracks_3d[i] for i, s in enumerate(steps)},
    kdims='step'
)
track_history.opts(
    opts.Scatter3D(c='black'),
    opts.Scatter3D('Hits.Noise', c='red'),
    opts.Path3D(color='black', show_legend=True),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(legend_position='right'),
    opts.NdLayout(fig_size=200)
).cols(4)

In [ ]:
hv.extension('plotly')
track_history = hv.HoloMap(
    {s: tracks_3d[i] for i, s in enumerate(steps)},
    kdims='step'
)
track_history.opts(
    opts.Scatter3D(size=2, color='black'),
    opts.Scatter3D('Hits.Noise', color='red'),
    opts.Path3D(color='black'),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(width=700, height=700)
)
hv.save(track_history, 'track_history_3d2.html', fmt='scrubber')

In [ ]:
# hv.extension('matplotlib')
# track_history = hv.HoloMap(
#     {s: tracks_projection[i] for i, s in enumerate(steps)},
#     kdims='step'
# )
# track_history.opts(
#     opts.Points(color='black'),
#     opts.Points('Hits.Noise', color='black'),
#     opts.Path(color='black', show_legend=True),
#     opts.Path('Tracks.fp', color='red'),
#     opts.Path('Tracks.fn', color='cyan'),
#     opts.Overlay(legend_position='right', fig_size=256),
# )

In [ ]:
hv.extension('bokeh')
track_history = hv.HoloMap(
    {s: tracks_projection[i] for i, s in enumerate(steps)},
    kdims='step'
)
track_history.opts(
    opts.Points(color='black', size=4),
    opts.Points('Hits.Noise', color='red'),
    opts.Path(color='black', show_legend=True),
    opts.Path('Tracks.fp', color='red'),
    opts.Path('Tracks.fn', color='cyan'),
    opts.Overlay(legend_position='right', width=700, height=700)
)


In [ ]:
hv.extension('bokeh')
track_history = hv.NdLayout(
    {s: tracks_by_track[i] for i, s in enumerate(steps)},
    kdims='step'
)
track_history.opts(
    opts.Graph(node_size=8, edge_color='act', colorbar=True),
    opts.EdgePaths(colorbar=True)
).cols(5)